## multiprocess

In [ ]:
from concurrent.futures import ProcessPoolExecutor, as_completed
from tqdm import tqdm
from coffea.nanoevents import NanoEventsFactory, NanoAODSchema
import time

def process_root_file(file_path):
    """处理单个ROOT文件并返回Muon值的统计信息或其它可以序列化的表示"""
    try:
        # 记录开始时间
        start_time = time.time()
        # 从ROOT文件创建NanoEventsArray
        events = NanoEventsFactory.from_root(file_path, treepath="Events", schemaclass=NanoAODSchema).events()
        # 提取Muon值并进行必要的处理，例如计算Muon数量
        # muon_count = len(events.Muon)
        muon_count = ak.Array(events.Muon.pt)
        # 记录结束时间
        end_time = time.time()
        # 返回结果和用时
        return muon_count, end_time - start_time
    except Exception as e:
        print(f"Error processing file {file_path}: {e}")
        return None, 0

from concurrent.futures import ProcessPoolExecutor, as_completed
from tqdm import tqdm
import input_file 
reload(input_file)

# 假设你有一个字典，key是样本名，value是对应的ROOT文件路径列表
samples_dict = input_file.input_file('test')# 1min


ABCD = 'A'
max_workers = 20  # 最大并行核心数

# 所有文件的Muon结果
all_muon_results = {}

# 初始化进度条
progress = tqdm(total=len(samples_dict['zg3_22']), desc='Processing files', unit='file')

with ProcessPoolExecutor(max_workers=max_workers) as executor:
    futures = []
    for sample_key, files in samples_dict.items():
        for file in files:
            future = executor.submit(process_root_file, file)
            futures.append((sample_key, future))
    
    for sample_key, future in futures:
        result, duration = future.result()
        if result is not None:
            if sample_key not in all_muon_results:
                all_muon_results[sample_key] = []
            all_muon_results[sample_key].append(result)
            # 更新进度条
            progress.update(1)
            progress.set_postfix({'last_duration': f'{duration:.2f}s', 'active_workers': f'{executor._max_workers}'})  # 显示上一个文件处理用时和活动核心数

progress.close()  # 关闭进度条

print(all_muon_results)

## multithread

In [ ]:
import input_file 
reload(input_file)
# dataC_ele_events = input_file.input_file('dataC_ele')# 15min
zg2_22_events = input_file.input_file('zg2_22')# 1min
zg3_22_events = input_file.input_file('zg3_22')# 1min
dy_22_events = input_file.input_file('dy_22')# 2min
EVENT_ELE = {
    'zg2_22': zg2_22_events,
    'zg3_22': zg3_22_events,
    'dy_22': dy_22_events,

}
def process_sample(sample, Tag, ABCD):
    #定义要用到的粒子
    Event = sample
    Electron = sample.Electron
    Photon = sample.Photon

    #罗列selection
    #electron
    ele_eta = (abs(Electron.eta) < 2.5) & ~((abs(Electron.eta) > 1.4442) & (abs(Electron.eta) < 1.566 ))
    ele_pt = (Electron.pt >= 20)
    ele_cutbased_id = (Electron.cutBased >= 3)

    #photon
    pho_pt = (Photon.pt >= 30)
    pho_eta = (abs(Photon.eta) < 2.5) & ~((abs(Photon.eta) > 1.4442) & (abs(Photon.eta) < 1.566))
    lg_near, lg_dr = Photon.nearest(Electron, axis=1, return_metric=True) #find the nearest electron near each photon
    pho_drlg = lg_dr>0.5
    pho_pixel = ~Photon.pixelSeed
    pho_genpartFlav = (Photon.genPartFlav == 1)
    # gen_photon = Event.GenPart[(Event.GenPart.pdgId==22) & Event.GenPart.hasFlags(['isLastCopy',"isPrompt"])]
    # genrecopho_near, genrecopho_dr = Photon.nearest(gen_photon, axis=1, return_metric=True)
    # pho_drrecogen = ak.fill_none(genrecopho_dr < 0.3, True)


    #联立选择条件
    ele_selection = ele_eta & ele_pt & ele_cutbased_id 
    photon_selection = pho_pt & pho_eta & pho_drlg 
    if 'data' in Tag:
        photon_selection = photon_selection  
    elif 'data' not in Tag and 'dy' not in Tag:
        photon_selection = photon_selection & pho_pixel & pho_genpartFlav 
    elif 'dy' in Tag:
        photon_selection = photon_selection & (~pho_pixel | ~pho_genpartFlav)
    else:
        print('Wrong sample!')


    #处理通过object挑选的事件 要求有两个electron，两个electron异号，最大的electron pt>=25
    good_electron = Electron[ele_selection][(ak.num(Electron[ele_selection])==2) & (ak.sum(Electron[ele_selection].pdgId,axis=1)==0) & (ak.fill_none(ak.max(Electron[ele_selection].pt, axis=1) >= 25, False)) & (ak.num(Photon[photon_selection])>=1)]
    good_event = Event[(ak.num(Electron[ele_selection])==2) & (ak.sum(Electron[ele_selection].pdgId,axis=1)==0) & (ak.fill_none(ak.max(Electron[ele_selection].pt, axis=1) >= 25, False)) & (ak.num(Photon[photon_selection])>=1)]
    good_photon = Photon[photon_selection][(ak.num(Electron[ele_selection])==2) & (ak.sum(Electron[ele_selection].pdgId,axis=1)==0) & (ak.fill_none(ak.max(Electron[ele_selection].pt, axis=1) >= 25, False)) & (ak.num(Photon[photon_selection])>=1)]


    #最终处理事例，调用函数，mass在质量窗内，且通过trigger
    #mass质量窗
    tem_ele = good_electron
    ele_vectors = ak.zip(
    {
    "pt": tem_ele["pt"],
    "eta": tem_ele["eta"],
    "phi": tem_ele["phi"],
    "mass": tem_ele["mass"],
    },
    with_name="PtEtaPhiMLorentzVector",
    )
    mass_ee = (ele_vectors[:, 0] + ele_vectors[:, 1]).mass

    #photon id
    pt_id = (good_photon.vidNestedWPBitmap[:] & 1) + 2*((good_photon.vidNestedWPBitmap[:] >> 1) & 1) 
    scEta_id = (good_photon.vidNestedWPBitmap[:] >> 2 & 1) + 2*((good_photon.vidNestedWPBitmap[:] >> 3) & 1) 
    HoE_id = (good_photon.vidNestedWPBitmap[:] >> 4 & 1) + 2*((good_photon.vidNestedWPBitmap[:] >> 5) & 1) 
    siesie_id = (good_photon.vidNestedWPBitmap[:] >> 6 & 1) + 2*((good_photon.vidNestedWPBitmap[:] >> 7) & 1) 
    chiso_id = (good_photon.vidNestedWPBitmap[:] >> 8 & 1) + 2*((good_photon.vidNestedWPBitmap[:] >> 9) & 1) 
    neuiso_id = (good_photon.vidNestedWPBitmap[:] >> 10 & 1) + 2*((good_photon.vidNestedWPBitmap[:] >> 11) & 1) 
    phoiso_id = (good_photon.vidNestedWPBitmap[:] >> 12 & 1) + 2*((good_photon.vidNestedWPBitmap[:] >> 13) & 1)
    if ABCD == "A":
        all_id = (pt_id == 3) & (scEta_id == 3) & (HoE_id == 3) & (siesie_id == 3) & (chiso_id == 3) & (neuiso_id == 3) & (phoiso_id == 3)
    elif ABCD == "B":
        all_id = (pt_id == 3) & (scEta_id == 3) & (HoE_id == 3) & (siesie_id == 3) & (chiso_id < 3) & (neuiso_id == 3) & (phoiso_id == 3)
    elif ABCD == "C":
        all_id = (pt_id == 3) & (scEta_id == 3) & (HoE_id == 3) & (siesie_id < 3) & (chiso_id == 3) & (neuiso_id == 3) & (phoiso_id == 3)
    elif ABCD == "D":
        all_id = (pt_id == 3) & (scEta_id == 3) & (HoE_id == 3) & (siesie_id < 3) & (chiso_id < 3) & (neuiso_id == 3) & (phoiso_id == 3)
    else:
        print('Wrong ABCD value!')

    selected_good_event = good_event[(good_event.HLT.Ele23_Ele12_CaloIdL_TrackIdL_IsoVL) & ((mass_ee >= 71) & (mass_ee <= 111)) & (ak.num(good_photon[all_id])>=1)]
    selected_good_electron = good_electron[(good_event.HLT.Ele23_Ele12_CaloIdL_TrackIdL_IsoVL) & ((mass_ee >= 71) & (mass_ee <= 111)) & (ak.num(good_photon[all_id])>=1)]
    selected_good_photon = good_photon[(good_event.HLT.Ele23_Ele12_CaloIdL_TrackIdL_IsoVL) & ((mass_ee >= 71) & (mass_ee <= 111)) & (ak.num(good_photon[all_id])>=1)]


    # 返回处理后的结果
    return selected_good_event, selected_good_electron, selected_good_photon

from concurrent.futures import ThreadPoolExecutor, as_completed
from tqdm import tqdm
import time
import threading  

# 假设你的样本字典名为samples_dict，每个key对应一个NanoEventsArray
samples_dict = EVENT_ELE  # 你的NanoEventsArray字典
ABCD= 'A'
max_workers = 50

# 使用ThreadPoolExecutor来并行处理
with ThreadPoolExecutor(max_workers=max_workers) as executor:
    # 提交任务到线程池
    futures = {}
    for key, nano_array_list in samples_dict.items():
        for idx, nano_array in enumerate(nano_array_list):
            # 记录任务开始时间
            start_time = time.time()
            future = executor.submit(process_sample, nano_array, key,ABCD)
            futures[future] = (key, idx, start_time)
    
    # 初始化进度条
    progress = tqdm(as_completed(futures), total=len(futures), desc='Processing samples', unit='sample')

    # 收集结果
    results = {key: [] for key in samples_dict}  # 初始化结果字典
    for future in progress:
        key, idx, start_time = futures[future]
        try:
            result = future.result()
            end_time = time.time()
            elapsed_time = end_time - start_time
            results[key].append((idx, result, elapsed_time))  # 将结果和处理时间存储到对应的key中
            # 更新进度条后缀，包括活动线程数
            active_threads = threading.active_count() - 1  # 减去主线程
            progress.set_postfix({'key': key, 'idx': idx, 'time': f'{elapsed_time:.2f}s', 'active_threads': f'{active_threads}/{max_workers}'})
        except Exception as exc:
            print(f'{key} array {idx} generated an exception: {exc}')
        progress.update(1)
            
    # 对结果进行排序，确保按照原NanoEventsArray的顺序
    for key in results:
        results[key].sort(key=lambda x: x[0])
        results[key] = [(res, time) for idx, res, time in results[key]]

In [1]:
import selection_cut
reload(selection_cut)
sample = EVENT['test'][0]
print('Start (2022E):',len(sample))
ele0 = selection_cut.electron_nTuple(sample,'mc')[0]
print('2e:',len(ele0),'Efficiency:',f"{len(ele0)/len(sample) * 100}%")
ele1 = selection_cut.electron_nTuple(sample,'mc')[1]
print('HLT:',len(ele1),'Efficiency:',f"{len(ele1)/len(ele0) * 100}%")
ele2 = selection_cut.electron_nTuple(sample,'mc')[2]
print('medium id:',len(ele2),'Efficiency:',f"{len(ele2)/len(ele1) * 100}%")
ele3 = selection_cut.electron_nTuple(sample,'mc')[3]
print('Eta:',len(ele3),'Efficiency:',f"{len(ele3)/len(ele2) * 100}%")
ele4 = selection_cut.electron_nTuple(sample,'mc')[4]
print('Pt:',len(ele4),'Efficiency:',f"{len(ele4)/len(ele3) * 100}%")
ele5 = selection_cut.electron_nTuple(sample,'mc')[5]
print('Opposite charge:',len(ele5),'Efficiency:',f"{len(ele5)/len(ele4) * 100}%")
ele6 = selection_cut.electron_nTuple(sample,'mc')[6]
print('Z mass window:',len(ele6),'Efficiency:',f"{len(ele6)/len(ele5) * 100}%")
pho1 = selection_cut.electron_nTuple(sample,'mc')[7]
print('Photon pt,eta,id,1 photon:',len(pho1),'Efficiency:',f"{len(pho1)/len(ele6) * 100}%")
selected_photons = selection_cut.electron_nTuple(sample,'mc')[8]
print('Photon pt,eta,id:',len(selected_photons),'Efficiency:',f"{len(selected_photons)/len(ele6) * 100}%")
pho2 = selection_cut.electron_nTuple(sample,'mc')[9]
print('Photon dR',len(pho2),'Efficiency:',f"{len(pho2)/len(pho1) * 100}%")
selected_photons2 = selection_cut.electron_nTuple(sample,'mc')[10]
print('Photon dR',len(selected_photons2),'Efficiency:',f"{len(selected_photons2)/len(selected_photons) * 100}%")
pho3 = selection_cut.electron_nTuple(sample,'mc')[11]
print('Photon prompt (Flav)',len(pho3),'Efficiency:',f"{len(pho3)/len(pho2) * 100}%")
selected_photons3 = selection_cut.electron_nTuple(sample,'mc')[12]
print('Photon prompt (Flav)',len(selected_photons3),'Efficiency:',f"{len(selected_photons3)/len(selected_photons2) * 100}%")
pho4 = selection_cut.electron_nTuple(sample,'mc')[13]
print('Photon prompt (dR)',len(pho4),'Efficiency:',f"{len(pho4)/len(pho3) * 100}%")
selected_photons4 = selection_cut.electron_nTuple(sample,'mc')[14]
print('Photon prompt (dR)',len(selected_photons4),'Efficiency:',f"{len(selected_photons4)/len(selected_photons3) * 100}%")


###ELECTRON

import selection_cut
reload(selection_cut)
sample = EVENT_ELE['dataE_ele'][0]
Tag = 'dataE'
# sample = EVENT_ELE['zg0'][0]
# Tag = 'zg0'
print('Start (2022E):',len(sample))
ele0 = selection_cut.electron_nTuple(sample,Tag)[0]
print('HLT:',len(ele0),'Efficiency:',f"{len(ele0)/len(sample) * 100}%")
ele1 = selection_cut.electron_nTuple(sample,Tag)[1]
ele1_ele = selection_cut.electron_nTuple(sample,Tag)[2]
print('pt, eta, medium id, charge, 2ele:',len(ele1),'Efficiency:',f"{len(ele1)/len(ele0) * 100}%")
ele2 = selection_cut.electron_nTuple(sample,Tag)[3]
ele2_ele = selection_cut.electron_nTuple(sample,Tag)[4]
print('Z mass window:',len(ele2),'Efficiency:',f"{len(ele2)/len(ele1) * 100}%")
# ele3 = selection_cut.electron_nTuple(sample,Tag)[3]
# print('Eta:',len(ele3),'Efficiency:',f"{len(ele3)/len(ele2) * 100}%")
# ele4 = selection_cut.electron_nTuple(sample,Tag)[4]
# print('Pt:',len(ele4),'Efficiency:',f"{len(ele4)/len(ele3) * 100}%")
# ele5 = selection_cut.electron_nTuple(sample,Tag)[5]
# print('Opposite charge:',len(ele5),'Efficiency:',f"{len(ele5)/len(ele4) * 100}%")
# ele6 = selection_cut.electron_nTuple(sample,Tag)[6]
# print('Z mass window:',len(ele6),'Efficiency:',f"{len(ele6)/len(ele5) * 100}%")
# pho1 = selection_cut.electron_nTuple(sample,Tag)[7]
# print('Photon pt,eta,id,1 photon:',len(pho1),'Efficiency:',f"{len(pho1)/len(ele6) * 100}%")
# selected_photons = selection_cut.electron_nTuple(sample,Tag)[8]
# print('Photon pt,eta,id:',len(selected_photons),'Efficiency:',f"{len(selected_photons)/len(ele6) * 100}%")
# pho2 = selection_cut.electron_nTuple(sample,Tag)[9]
# print('Photon dR',len(pho2),'Efficiency:',f"{len(pho2)/len(pho1) * 100}%")
# selected_photons2 = selection_cut.electron_nTuple(sample,Tag)[10]
# print('Photon dR',len(selected_photons2),'Efficiency:',f"{len(selected_photons2)/len(selected_photons) * 100}%")
# pho3 = selection_cut.electron_nTuple(sample,Tag)[11]
# print('Photon prompt (Flav)',len(pho3),'Efficiency:',f"{len(pho3)/len(pho2) * 100}%")
# selected_photons3 = selection_cut.electron_nTuple(sample,Tag)[12]
# print('Photon prompt (Flav)',len(selected_photons3),'Efficiency:',f"{len(selected_photons3)/len(selected_photons2) * 100}%")
# pho4 = selection_cut.electron_nTuple(sample,Tag)[13]
# print('Photon prompt (dR)',len(pho4),'Efficiency:',f"{len(pho4)/len(pho3) * 100}%")
# selected_photons4 = selection_cut.electron_nTuple(sample,Tag)[14]
# print('Photon prompt (dR)',len(selected_photons4),'Efficiency:',f"{len(selected_photons4)/len(selected_photons3) * 100}%")







NameError: name 'reload' is not defined

In [ ]:
### MUON
import selection_cut
reload(selection_cut)
sample = EVENT_MUON['dataE_muon'][0]
Tag = 'dataE'
# sample = EVENT_MUON['zg0'][0]
# Tag = 'zg0'
print('Start (2022E):',len(sample))
mu0 = selection_cut.muon_nTuple(sample,Tag)[0]
print('2mu:',len(mu0),'Efficiency:',f"{len(mu0)/len(sample) * 100}%")
mu1 = selection_cut.muon_nTuple(sample,Tag)[1]
print('HLT:',len(mu1),'Efficiency:',f"{len(mu1)/len(mu0) * 100}%")
mu2 = selection_cut.muon_nTuple(sample,Tag)[2]
print('tight id:',len(mu2),'Efficiency:',f"{len(mu2)/len(mu1) * 100}%")
mu5 = selection_cut.muon_nTuple(sample,Tag)[3]
print('Opposite charge:',len(mu5),'Efficiency:',f"{len(mu5)/len(mu2) * 100}%")
mu6 = selection_cut.muon_nTuple(sample,Tag)[4]
print('tight iso,Pt, Eta:',len(mu6),'Efficiency:',f"{len(mu6)/len(mu5) * 100}%")
mu7 = selection_cut.muon_nTuple(sample,Tag)[5]
print('Z boson mass window:',len(mu7),'Efficiency:',f"{len(mu7)/len(mu6) * 100}%")
pho1 = selection_cut.muon_nTuple(sample,Tag)[6]
print('Photon pt,eta,id,1 photon:',len(pho1),'Efficiency:',f"{len(pho1)/len(ele6) * 100}%")
selected_photons = selection_cut.muon_nTuple(sample,Tag)[7]
print('Photon pt,eta,id:',len(selected_photons),'Efficiency:',f"{len(selected_photons)/len(ele6) * 100}%")
pho2 = selection_cut.muon_nTuple(sample,Tag)[8]
print('Photon dR',len(pho2),'Efficiency:',f"{len(pho2)/len(pho1) * 100}%")
selected_photons2 = selection_cut.muon_nTuple(sample,Tag)[9]
print('Photon dR',len(selected_photons2),'Efficiency:',f"{len(selected_photons2)/len(selected_photons) * 100}%")
pho3 = selection_cut.muon_nTuple(sample,Tag)[10]
print('Photon prompt (Flav)',len(pho3),'Efficiency:',f"{len(pho3)/len(pho2) * 100}%")
selected_photons3 = selection_cut.muon_nTuple(sample,Tag)[11]
print('Photon prompt (Flav)',len(selected_photons3),'Efficiency:',f"{len(selected_photons3)/len(selected_photons2) * 100}%")
pho4 = selection_cut.muon_nTuple(sample,Tag)[12]
print('Photon prompt (dR)',len(pho4),'Efficiency:',f"{len(pho4)/len(pho3) * 100}%")
selected_photons4 = selection_cut.muon_nTuple(sample,Tag)[13]
print('Photon prompt (dR)',len(selected_photons4),'Efficiency:',f"{len(selected_photons4)/len(selected_photons3) * 100}%")


In [ ]:
import awkward as ak
import uproot
import numpy as np
from coffea.lumi_tools import LumiMask

def run_num(sample,Tag):
    # if Tag == 'C':
    #     mask = LumiMask('json_file/Cert_Collisions2022_eraC_355862_357482_Golden.json')
    #     good_data = mask(sample['run'], sample['luminosityBlock'])
    #     return(sample[good_data]) 
    # if Tag == 'D':
    #     mask = LumiMask('json_file/Cert_Collisions2022_eraD_357538_357900_Golden.json')
    #     good_data = mask(sample['run'], sample['luminosityBlock'])
    #     return(sample[good_data])        
    if Tag == 'E':
        mask = LumiMask('json_file/Cert_Collisions2022_eraE_359022_360331_Golden.json')
        good_data = mask(sample['run'], sample['luminosityBlock'])
        return(sample[good_data])
    # if Tag == 'F':
    #     mask = LumiMask('json_file/Cert_Collisions2022_eraF_360390_362167_Golden.json')
    #     good_data = mask(sample['run'], sample['luminosityBlock'])
    #     return(sample[good_data])
    if Tag == 'G':
        mask = LumiMask('json_file/Cert_Collisions2022_eraG_362433_362760_Golden.json')
        good_data = mask(sample['run'], sample['luminosityBlock'])
        return(sample[good_data])

def electron_nTuple(sample,Tag):
# ele
    pass_ele = (sample.Electron.pt > 10)
    ele0 = sample[(ak.num(sample.Electron.pt[pass_ele],axis=1)==2)]
    ele1 = ele0[ele0.HLT.Ele23_Ele12_CaloIdL_TrackIdL_IsoVL]#
    ele2 = ele1[ak.all(ele1.Electron.cutBased>=3,axis=1)]#0:fail, 1:veto, 2:loose, 3:medium, 4:tight
    ele3 = ele2[ak.all(abs(ele2.Electron.eta) < 2.4,axis=1) & ak.all(~((abs(ele2.Electron.eta) > 1.4442) & (abs(ele2.Electron.eta) < 1.566)),axis=1)]
    pt = ele3.Electron.pt
    filtered_pt = [(ak.num(pt) == 2) & (ak.min(pt[:, :2], axis=1) >= 20) & (ak.max(pt[:, :2], axis=1) >= 25)]
    ele4 = ele3[filtered_pt[0]]
    pid_cut = (ak.sum(ele4.Electron.pdgId,axis=1)==0) & (ak.num(ele4.Electron.pdgId,axis=1)==2)
    filled_pid_cut = ak.where(ak.is_none(pid_cut), False, pid_cut)
    ele5 = ele4[filled_pid_cut]
    tem_ele = ele5.Electron
    ele_vectors = ak.zip(
    {
        "pt": tem_ele["pt"],
        "eta": tem_ele["eta"],
        "phi": tem_ele["phi"],
        "mass": tem_ele["mass"],
    },
    with_name="PtEtaPhiMLorentzVector",
    )
    mass_ee = (ele_vectors[:, 0] + ele_vectors[:, 1]).mass
    ele6 = ele5[(mass_ee >= 71) & (mass_ee <= 111)]
#photon
    pt_cut = ele6.Photon[ele6.Photon.pt > 30]
    eta_cut = pt_cut[(abs(pt_cut.eta) < 2.5) & ~((abs(pt_cut.eta) > 1.4442) & (abs(pt_cut.eta) < 1.566))]
    pt_id = (eta_cut.vidNestedWPBitmap[:] & 1) + 2*((eta_cut.vidNestedWPBitmap[:] >> 1) & 1) 
    scEta_id = (eta_cut.vidNestedWPBitmap[:] >> 2 & 1) + 2*((eta_cut.vidNestedWPBitmap[:] >> 3) & 1) 
    HoE_id = (eta_cut.vidNestedWPBitmap[:] >> 4 & 1) + 2*((eta_cut.vidNestedWPBitmap[:] >> 5) & 1) 
    siesie_id = (eta_cut.vidNestedWPBitmap[:] >> 6 & 1) + 2*((eta_cut.vidNestedWPBitmap[:] >> 7) & 1) 
    chiso_id = (eta_cut.vidNestedWPBitmap[:] >> 8 & 1) + 2*((eta_cut.vidNestedWPBitmap[:] >> 9) & 1) 
    neuiso_id = (eta_cut.vidNestedWPBitmap[:] >> 10 & 1) + 2*((eta_cut.vidNestedWPBitmap[:] >> 11) & 1) 
    phoiso_id = (eta_cut.vidNestedWPBitmap[:] >> 12 & 1) + 2*((eta_cut.vidNestedWPBitmap[:] >> 13) & 1)
    all_id_3 = (pt_id == 3) & (scEta_id == 3) & (HoE_id == 3) & (siesie_id == 3) & (chiso_id == 3) & (neuiso_id == 3) & (phoiso_id == 3)
    id_cut = eta_cut[all_id_3]
    single_photon_per_event = ak.num(id_cut) == 1
    pho1 = ele6[single_photon_per_event]
    selected_photons = id_cut[single_photon_per_event]
    lg_near, lg_dr = selected_photons.nearest(pho1.Muon, axis=1, return_metric=True)
    pho2 = pho1[ak.num(selected_photons[lg_dr>0.5])==1]
    selected_photons2 = selected_photons[ak.num(selected_photons[lg_dr>0.5])==1]
    if 'data' in Tag:
        # return pho2,selected_photons2
        return ele0,ele1,ele2,ele3,ele4,ele5,ele6,pho1,selected_photons 
    if 'data' not in Tag:
        prompt_cut = (selected_photons2.genPartFlav == 1)
        pho3 = pho2[ak.num(selected_photons2[prompt_cut])==1]
        selected_photons3 = selected_photons2[ak.num(selected_photons2[prompt_cut])==1]
        gen_photon = pho3.GenPart[(pho3.GenPart.pdgId==22) & pho3.GenPart.hasFlags(['isLastCopy',"isPrompt"])]
        photon4_near, photon4_gen_photon_dr = selected_photons3.nearest(gen_photon, axis=1, return_metric=True)
        mask = ak.fill_none(photon4_gen_photon_dr < 0.3, True)
        selected_photons4 = selected_photons3[mask]
        pho4 = pho3[ak.num(selected_photons4)==1]
        # return pho4,selected_photons4
        return ele0,ele1,ele2,ele3,ele4,ele5,ele6,pho1,selected_photons,pho2,selected_photons2,pho3,selected_photons3,pho4,selected_photons4 #fsr



    

def muon_nTuple(sample,Tag):
    pass_mu = (sample.Muon.pt > 10) & (sample.Muon.looseId) & (sample.Muon.pfIsoId >= 2)
    # pass_mu = (sample.Muon.pt > 10)
    mu0 = sample[ak.num(sample.Muon[pass_mu],axis=1)==2]
    mu1 = mu0[mu0.HLT.Mu17_TrkIsoVVL_Mu8_TrkIsoVVL_DZ_Mass3p8]
    mu2 = mu1.Muon[mu1.Muon.tightId]
    mu3 = mu2[mu2.pfIsoId >= 4]
    mu4 = mu3[abs(mu3.eta) < 2.4]
    pt = mu4.pt
    filtered_pt = [(ak.num(pt) >= 2) & (ak.min(pt[:, :2], axis=1) >= 20) & (ak.max(pt[:, :2], axis=1) >= 25)]
    mu5 = mu1[filtered_pt[0]]
    pid_cut = (ak.sum(mu5.Muon.pdgId,axis=1)==0) & (ak.num(mu5.Muon.pdgId,axis=1)==2)
    filled_pid_cut = ak.where(ak.is_none(pid_cut), False, pid_cut)
    mu6 = mu5[filled_pid_cut]
    tem_Muon = mu6.Muon[ak.num(mu6.Muon.tightId) == 2]
    muon_vectors = ak.zip(
    {
        "pt": tem_Muon["pt"],
        "eta": tem_Muon["eta"],
        "phi": tem_Muon["phi"],
        "mass": tem_Muon["mass"],
    },
    with_name="PtEtaPhiMLorentzVector",
    )
    mass_mumu = (muon_vectors[:, 0] + muon_vectors[:, 1]).mass
    mu7 = mu6[(mass_mumu >= 71) & (mass_mumu <= 111)]
#photon
    pt_cut = mu7.Photon[mu7.Photon.pt > 30]
    eta_cut = pt_cut[(abs(pt_cut.eta) < 2.5) & ~((abs(pt_cut.eta) > 1.4442) & (abs(pt_cut.eta) < 1.566))]
    pt_id = (eta_cut.vidNestedWPBitmap[:] & 1) + 2*((eta_cut.vidNestedWPBitmap[:] >> 1) & 1) 
    scEta_id = (eta_cut.vidNestedWPBitmap[:] >> 2 & 1) + 2*((eta_cut.vidNestedWPBitmap[:] >> 3) & 1) 
    HoE_id = (eta_cut.vidNestedWPBitmap[:] >> 4 & 1) + 2*((eta_cut.vidNestedWPBitmap[:] >> 5) & 1) 
    siesie_id = (eta_cut.vidNestedWPBitmap[:] >> 6 & 1) + 2*((eta_cut.vidNestedWPBitmap[:] >> 7) & 1) 
    chiso_id = (eta_cut.vidNestedWPBitmap[:] >> 8 & 1) + 2*((eta_cut.vidNestedWPBitmap[:] >> 9) & 1) 
    neuiso_id = (eta_cut.vidNestedWPBitmap[:] >> 10 & 1) + 2*((eta_cut.vidNestedWPBitmap[:] >> 11) & 1) 
    phoiso_id = (eta_cut.vidNestedWPBitmap[:] >> 12 & 1) + 2*((eta_cut.vidNestedWPBitmap[:] >> 13) & 1)
    all_id_3 = (pt_id == 3) & (scEta_id == 3) & (HoE_id == 3) & (siesie_id == 3) & (chiso_id == 3) & (neuiso_id == 3) & (phoiso_id == 3)
    id_cut = eta_cut[all_id_3]
    single_photon_per_event = ak.num(id_cut) == 1
    pho1 = mu7[single_photon_per_event]
    selected_photons = id_cut[single_photon_per_event]
    lg_near, lg_dr = selected_photons.nearest(pho1.Muon, axis=1, return_metric=True)
    pho2 = pho1[ak.num(selected_photons[lg_dr>0.5])==1]
    selected_photons2 = selected_photons[ak.num(selected_photons[lg_dr>0.5])==1]
    if 'data' in Tag:
        return mu0,mu1,mu2,mu5,mu6,mu7,pho1,selected_photons
    if 'data' not in Tag:
        prompt_cut = (selected_photons2.genPartFlav == 1)
        pho3 = pho2[ak.num(selected_photons2[prompt_cut])==1]
        selected_photons3 = selected_photons2[ak.num(selected_photons2[prompt_cut])==1]
        gen_photon = pho3.GenPart[(pho3.GenPart.pdgId==22) & pho3.GenPart.hasFlags(['isLastCopy',"isPrompt"])]
        photon4_near, photon4_gen_photon_dr = selected_photons3.nearest(gen_photon, axis=1, return_metric=True)
        mask = ak.fill_none(photon4_gen_photon_dr < 0.3, True)
        selected_photons4 = selected_photons3[mask]
        pho4 = pho3[ak.num(selected_photons4)==1]
        return mu0,mu1,mu2,mu5,mu6,mu7,pho1,selected_photons,pho2,selected_photons2,pho3,selected_photons3,pho4,selected_photons4

    

def VARIABLES(Tag,key,pho_ele,electron,mass_ee,mass_gee,event_number):
    if Tag == 'ele':
        VARIABLES = {
    ##pho
                'photon_pt':pho_ele.pt,
                'photon_eta':pho_ele.eta,
    ##muon
                'ele1_pt':electron.Electron[:,0].pt,
                'ele2_pt':electron.Electron[:,1].pt,
                'ele1_eta':electron.Electron[:,0].eta,
                'ele2_eta':electron.Electron[:,1].eta,
                'ele_mass':mass_ee,
                'gee_mass':mass_gee,            
            }

        # VARIABLES.update(out_VARIABLES)
        if 'data' not in key:
            # uncertainty = {
            #     'LHEPdfWeight':electron.LHEPdfWeight,
            #     'LHEScaleWeight':electron.LHEScaleWeight,
            #     'PSWeight':electron.PSWeight,
            # }
            # VARIABLES.update(uncertainty)
            dict_genweight = {'generator_weight':np.sign(electron.Generator.weight)}
            VARIABLES.update(dict_genweight)
            VARIABLES.update(event_number)
    return(VARIABLES)


def MASS_Muon(samples,tag):
    tem_Muon = samples.Muon
    muon_vectors = ak.zip(
    {
        "pt": tem_Muon["pt"],
        "eta": tem_Muon["eta"],
        "phi": tem_Muon["phi"],
        "mass": tem_Muon["mass"],
    },
    with_name="PtEtaPhiMLorentzVector",
    )
    tem_Photon = samples.Photon[:, 0]
    photon_vectors = ak.zip(
    {
        "pt": tem_Photon["pt"],
        "eta": tem_Photon["eta"],
        "phi": tem_Photon["phi"],
        "mass": tem_Photon.mass,
    },
    with_name="PtEtaPhiMLorentzVector",
    )
    if tag == 'mu':
        invariant_mass = (muon_vectors[:, 0] + muon_vectors[:, 1]).mass
        return(invariant_mass)
    if tag == 'gmumu':
        invariant_mass = (muon_vectors[:, 0] + muon_vectors[:, 1] + photon_vectors).mass
        return(invariant_mass)

def MASS_Ele(pho_ele,electron,tag):
    tem_Ele = electron.Electron
    ele_vectors = ak.zip(
    {
        "pt": tem_Ele["pt"],
        "eta": tem_Ele["eta"],
        "phi": tem_Ele["phi"],
        "mass": tem_Ele["mass"],
    },
    with_name="PtEtaPhiMLorentzVector",
    )
    tem_Photon = pho_ele
    photon_vectors = ak.zip(
    {
        "pt": tem_Photon["pt"],
        "eta": tem_Photon["eta"],
        "phi": tem_Photon["phi"],
        "mass": tem_Photon.mass,
    },
    with_name="PtEtaPhiMLorentzVector",
    )
    if tag == 'ee':
        invariant_mass = (ele_vectors[:, 0] + ele_vectors[:, 1]).mass
        return(invariant_mass)
    if tag == 'gee':
        invariant_mass = (ele_vectors[:, 0] + ele_vectors[:, 1] + photon_vectors).mass
        return(invariant_mass)

def gen_mass(samples, tag):
    gen_muon_vectors = ak.zip(
        {
            "pt": samples['GenDressedLepton']['pt'],
            "eta": samples['GenDressedLepton']['eta'],
            "phi": samples['GenDressedLepton']['phi'],
            "mass": samples['GenDressedLepton']['mass'],
        },
        with_name="PtEtaPhiMLorentzVector"
    )
    # GEN_gamma = GEN[GEN['pdgId'] == 22]
    gen_photon_vector = ak.zip(
        {
            "pt": samples['GenIsolatedPhoton']['pt'],
            "eta": samples['GenIsolatedPhoton']['eta'],
            "phi": samples['GenIsolatedPhoton']['phi'],
            "mass": samples['GenIsolatedPhoton']['mass'],
        },
        with_name="PtEtaPhiMLorentzVector"
    )
    # gen_muon_vectors = gen_muon_vectors_0[ak.num(gen_muon_vectors_0)==2]
    # gen_photon_vector = gen_photon_vector_0[ak.num(gen_muon_vectors_0)==2]
    if tag == 'mu':
        invariant_mass = (gen_muon_vectors[:, 0] + gen_muon_vectors[:, 1]).mass
        return(invariant_mass)
    if tag == 'gmumu':
        invariant_mass = (gen_muon_vectors[:, 0] + gen_muon_vectors[:, 1] + gen_photon_vector).mass
        return(invariant_mass)





def DR(obj_A, obj_B, drmin=0.3):
    ## Method 2
    objB_near, objB_dr = obj_A.nearest(obj_B, return_metric=True)
    mask = ak.fill_none(objB_dr > drmin, True) # I guess to use True is because if there are no objB, all the objA are clean
    return (mask)
